## This file will navigate to a directory with multiple CSVs and compile ALL of them into 1 master CSV, with the header

In [1]:
from tqdm.notebook import tqdm

In [4]:
# import dependencies
import pandas as pd
import os
import glob

path = 'Data/Geocoded/'
path_files = glob.glob(os.path.join(path, "*.csv"))
joined_df = pd.concat((pd.read_csv(f) for f in path_files)).drop_duplicates().reset_index(drop=True)
joined_df = joined_df.drop(columns='url')
joined_df = joined_df.loc[:, ~joined_df.columns.str.startswith('Unnamed')]

joined_df.to_csv('cobbGeocoded.csv')
print('complete!')

complete!


In [ ]:
# clean up
joined_df = joined_df[[
    'Street Name', 
    'Sale Date',
    'Sale Price',
    'Square Ft ']]

joined_df['full_address'] = joined_df['Street Name'] + ' Cobb County, GA'

# Function to convert currency string to integer
def convert_currency_to_int(currency):
    currency = currency.replace('$', '').replace(',', '')  # Remove '$' and ','
    currency = currency.split('.')[0]
    return int(currency)

# Apply the conversion function to the 'Sale Price' column and create a new 'price_num' column
joined_df['price_num'] = joined_df['Sale Price'].apply(convert_currency_to_int)

# create price / SF columns
joined_df['price_sf'] = joined_df['price_num'] / joined_df['Square Ft ']

joined_df.rename(columns={
    'Sale Date':'sale_date',
    'Sale Price':'sale_price',
    'Square Ft ':'sq_feet',
    'Street Name':'street_name'
}, inplace=True)

joined_df = joined_df[[
    'street_name',
    'full_address',
    'sale_date',
    'sale_price',
    'price_num',
    'sq_feet',
    'price_sf'
]]

joined_df['sale_date']= pd.to_datetime(joined_df['sale_date'])

joined_df = joined_df.sort_values(by=['sale_date'])

In [41]:
# split the big dataset into smaller chunks
n = 15000  #chunk row size
list_df = [joined_df[i:i+n] for i in range(0,joined_df.shape[0],n)]

list_df[0].to_csv('Data/Ready_4_geocode/cobb_batch1.csv')
list_df[1].to_csv('Data/Ready_4_geocode/cobb_batch2.csv')
list_df[2].to_csv('Data/Ready_4_geocode/cobb_batch3.csv')
list_df[3].to_csv('Data/Ready_4_geocode/cobb_batch4.csv')
list_df[4].to_csv('Data/Ready_4_geocode/cobb_batch5.csv')
list_df[5].to_csv('Data/Ready_4_geocode/cobb_batch6.csv')

print('complete!')

complete!
